In [7]:
import os
import dotenv
import pandas as pd
import geopandas as gpd
import pyeuropeana.apis as apis
import pyeuropeana.utils as utils

In [2]:
dotenv.load_dotenv()

True

In [3]:
print(os.getenv('EUROPEANA_API_KEY'))

snetrourst


In [4]:
def get_provider_data(provider_name, world_gdf, rows=1000):
    """
    Query Europeana API for a specific data provider and return a processed DataFrame.
    
    Args:
    provider_name (str): Name of the data provider to query
    world_gdf (GeoDataFrame): World map GeoDataFrame for spatial join
    rows (int): Number of rows to retrieve (default 1000)
    
    Returns:
    pandas.DataFrame: Processed DataFrame with the queried data, including object location
    """
    # Query the Europeana API
    myquery = apis.search(
        query='pl_wgs84_pos_lat:(*)',
        qf=f'DATA_PROVIDER:"{provider_name}"',
        rows=rows
    )
    
    # Create initial DataFrame
    myquery_df = pd.DataFrame(myquery["items"], columns=['edmPlaceLatitude', 'edmPlaceLongitude', 'id', 'country', 'dataProvider', 'dcCreator'])
    
    # Function to extract single value from list or return original value
    def extract_single(x):
        return x[0] if isinstance(x, list) and len(x) > 0 else x
    
    # Apply extraction to relevant columns
    for col in ['edmPlaceLatitude', 'edmPlaceLongitude', 'country', 'dataProvider', 'dcCreator']:
        myquery_df[col] = myquery_df[col].apply(extract_single)
    
    # Convert latitude and longitude to float type
    myquery_df['edmPlaceLatitude'] = pd.to_numeric(myquery_df['edmPlaceLatitude'], errors='coerce')
    myquery_df['edmPlaceLongitude'] = pd.to_numeric(myquery_df['edmPlaceLongitude'], errors='coerce')
    
    # Create a GeoDataFrame from the DataFrame
    gdf = gpd.GeoDataFrame(
        myquery_df, 
        geometry=gpd.points_from_xy(myquery_df.edmPlaceLongitude, myquery_df.edmPlaceLatitude), 
        crs="EPSG:4326"
    )

    # Perform spatial join
    gdf_with_country = gpd.sjoin(gdf, world_gdf[['geometry', 'name']], how='left', op='within')

    # Add the new column to the original DataFrame
    myquery_df['object_location'] = gdf_with_country['name']

    # Fill NaN values (points that don't fall within any country) with "Unknown"
    myquery_df['object_location'] = myquery_df['object_location'].fillna("Unknown")
    
    return myquery_df

In [8]:
# Load world map dataset
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

/tmp/ipykernel_135846/1808957771.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [5]:
df = get_provider_data("Museum of World Culture")
print(df.head())

   edmPlaceLatitude  edmPlaceLongitude                              id  \
0              51.0               10.0  /91670/SMVK_VKM_arkiv_15360114   
1              47.0                2.0  /91670/SMVK_VKM_arkiv_15360097   
2              47.0                2.0  /91670/SMVK_VKM_arkiv_15360049   
3              47.0                2.0  /91670/SMVK_VKM_arkiv_15359964   
4             -14.0              -53.0  /91670/SMVK_VKM_arkiv_15355968   

  country             dataProvider             dcCreator  
0  Sweden  Museum of World Culture        Morner, Birger  
1  Sweden  Museum of World Culture        Morner, Birger  
2  Sweden  Museum of World Culture        Morner, Birger  
3  Sweden  Museum of World Culture        Morner, Birger  
4  Sweden  Museum of World Culture  Ihering, Hermann von  


In [6]:
df

,edmPlaceLatitude,edmPlaceLongitude,id,country,dataProvider,dcCreator
0,51.00000,10.00000,/91670/SMVK_VKM_arkiv_15360114,Sweden,Museum of World Culture,"Morner, Birger"
1,47.00000,2.00000,/91670/SMVK_VKM_arkiv_15360097,Sweden,Museum of World Culture,"Morner, Birger"
2,47.00000,2.00000,/91670/SMVK_VKM_arkiv_15360049,Sweden,Museum of World Culture,"Morner, Birger"
3,47.00000,2.00000,/91670/SMVK_VKM_arkiv_15359964,Sweden,Museum of World Culture,"Morner, Birger"
4,-14.00000,-53.00000,/91670/SMVK_VKM_arkiv_15355968,Sweden,Museum of World Culture,"Ihering, Hermann von"
...,...,...,...,...,...,...
995,-17.05687,-64.99123,/91609/SMVK_VKM_fotografi_2606292,Sweden,Museum of World Culture,"Rosen, Eric von"
996,-17.05687,-64.99123,/91609/SMVK_VKM_fotografi_2606291,Sweden,Museum of World Culture,"Rosen, Eric von"
997,-17.05687,-64.99123,/91609/SMVK_VKM_fotografi_2606290,Sweden,Museum of World Culture,"Rosen, Eric von"
998,-17.05687,-64.99123,/91609/SMVK_VKM_fotografi_2606289,Sweden,Museum of World Culture,"Rosen, Eric von"
